# APAN 5400 Group 10 DEMO

# Load data from `API`, and select columns that we needed
- The `API` have Limitations of `1000` records retrieved at a time, we using while loop to retrieved all the records and appended it to our json file.

In [1]:
import requests
import json

# API endpoint URL
url = 'https://data.ny.gov/resource/7rrd-248n.json?$select=:*,fuel_type_code,station_name,street_address,city,state,zip,station_phone,status_code,access_days_time,ev_level1_evse_num,ev_level2_evse_num,ev_dc_fast_count,ev_network,ev_connector_types&$limit=1000'

# GET request to the API endpoint URL
response = requests.get(url)

# Extract JSON data from the response
json_data = response.json()

# Retrieve additional pages of data and append to the JSON file
offset = 1000
while len(json_data) % 1000 == 0:
    # Increment the offset parameter to retrieve the next page of data
    next_url = f'https://data.ny.gov/resource/7rrd-248n.json?$select=:*,fuel_type_code,station_name,street_address,city,state,zip,station_phone,status_code,access_days_time,ev_level1_evse_num,ev_level2_evse_num,ev_dc_fast_count,ev_network,ev_connector_types&$limit=1000&$offset={offset}'
    next_response = requests.get(next_url)
    next_json_data = next_response.json()
    json_data.extend(next_json_data)
    offset += 1000

# Write the JSON data to a file
with open('EV_Charging_Station.json', 'w') as f:
    json.dump(json_data, f)

In [2]:
# Load the JSON data from the file
with open('EV_Charging_Station.json', 'r') as f:
    json_data = json.load(f)

# Check the number of records in the JSON data
num_records = len(json_data)
print(f'Number of records: {num_records}')

Number of records: 3314


In [3]:
json_data[0:3]

[{':id': 'row-ue73_sg9i~33wf',
  ':created_at': '2023-04-23T19:00:49.547Z',
  ':updated_at': '2023-04-23T19:00:56.553Z',
  ':version': 'rv-bdde.dq6e-vhdt',
  'fuel_type_code': 'ELEC',
  'station_name': 'Turner Parking Ramp',
  'street_address': '1 Perkins Dr.',
  'city': 'Buffalo',
  'state': 'NY',
  'zip': '14202',
  'station_phone': '866-816-7584',
  'status_code': 'E',
  'ev_level2_evse_num': '4.0',
  'ev_network': 'EV Connect',
  'ev_connector_types': 'J1772'},
 {':id': 'row-dzf9_3wms.mpyj',
  ':created_at': '2023-04-23T19:00:49.547Z',
  ':updated_at': '2023-04-23T19:00:56.553Z',
  ':version': 'rv-qa9t~rwqh.xhbk',
  'fuel_type_code': 'ELEC',
  'station_name': 'Niagara Power Vista',
  'street_address': '5777 Lewiston Rd',
  'city': 'Lewiston',
  'state': 'NY',
  'zip': '14092',
  'status_code': 'E',
  'access_days_time': '24 hours daily',
  'ev_level2_evse_num': '6.0',
  'ev_network': 'Non-Networked',
  'ev_connector_types': 'J1772'},
 {':id': 'row-p2f5~263m-viei',
  ':created_at': 

# Connect to MongoDB

In [4]:
from pymongo import MongoClient

# Creating a MongoDB client and connecting to the database
client = MongoClient('localhost',27017) ## or MongoClient("localhost:27017")
db = client.apan5400

# Create MongoDB Collection from JSON file

In [5]:
with open('EV_Charging_Station.json', 'r') as f:
    data = json.load(f)
    
# Create a collection and insert the data
collection = db['APAN5400_Group_Demo']
collection.drop()
collection = db['APAN5400_Group_Demo']
collection.insert_many(data)

# Find a random document
collection.find_one()

{'_id': ObjectId('644991cd73091b5346905dcb'),
 ':id': 'row-ue73_sg9i~33wf',
 ':created_at': '2023-04-23T19:00:49.547Z',
 ':updated_at': '2023-04-23T19:00:56.553Z',
 ':version': 'rv-bdde.dq6e-vhdt',
 'fuel_type_code': 'ELEC',
 'station_name': 'Turner Parking Ramp',
 'street_address': '1 Perkins Dr.',
 'city': 'Buffalo',
 'state': 'NY',
 'zip': '14202',
 'station_phone': '866-816-7584',
 'status_code': 'E',
 'ev_level2_evse_num': '4.0',
 'ev_network': 'EV Connect',
 'ev_connector_types': 'J1772'}

In [6]:
# Count documents in the collection
total_docs = collection.count_documents({})
print(f'Number of documents: {total_docs}')

Number of documents: 3314


# Query example
- Print the City `Bufflo`, return `address`, `station name` and `phone number`
- Count the `total number of charging stations` within the `city`

In [7]:
results = db.APAN5400_Group_Demo.find({"city":'Buffalo'})

for i, doc in  enumerate(results, start=1):
    print(f"{i}.    Address: {doc['street_address']}, {doc['city']},{doc['state']} \n Station name: {doc['station_name']}: {doc['station_phone']}")

1.    Address: 1 Perkins Dr., Buffalo,NY 
 Station name: Turner Parking Ramp: 866-816-7584
2.    Address: 960 Busti Ave, Buffalo,NY 
 Station name: 960 BUSTI 960 BUSTI 1: 888-758-4389
3.    Address: Oak St, Buffalo,NY 
 Station name: ERIE COUNTY ECC STATION 1: 888-758-4389
4.    Address: 683 Northland Ave, Buffalo,NY 
 Station name: NORTHLAND 2: 888-758-4389
5.    Address: 1300 Elmwood Ave., Buffalo,NY 
 Station name: BUFFALO STATE EV STATION 1: 888-758-4389
6.    Address: 1615 Amherst Manor Dr, Buffalo,NY 
 Station name: NORTHTOWN CENTE STATION 2: 888-758-4389
7.    Address: 1138 Hertel Ave, Buffalo,NY 
 Station name: Tringali's: 866-816-7584
8.    Address: 589 Ellicott St, Buffalo,NY 
 Station name: 854ELLICOTT 589ELLICOTT3 CS: 888-758-4389
9.    Address: 1197 Niagara Street, Buffalo,NY 
 Station name: Rich Products: 866-816-7584
10.    Address: 185 Hayes Rd, Buffalo,NY 
 Station name: SUNY BUFFALO PARKER 7: 888-758-4389
11.    Address: BRL Parking Lot 3rd from Building  1 South St, 

In [8]:
pipeline = [ 
            {"$match": {"city": "Buffalo"}},
            {"$group": {"_id": None, "count": {"$sum": 1}}}
]

result = db.APAN5400_Group_Demo.aggregate(pipeline)

for doc in result:
    print(f"Total number of charging station {doc['count']}.")

Total number of charging station 134.


# Find the number of charging station based on `zipcode`

In [9]:
pipeline = [
            {"$match": {"zip": "14624" } },
            {"$group": {"_id": None, "count": { "$sum": 1 }}}
]

result = db.APAN5400_Group_Demo.aggregate(pipeline)

for doc in result:
    print(f"Total number of charging station {doc['count']}.")

Total number of charging station 5.


# Create `HTML`

In [10]:
html = """
<!DOCTYPE html>
<html>
<head>
 <title>EV Charging Station Search Engine</title>
 <style>
  body {
   background-image: url("{{ bg_image_url }}");
   background-size: cover;
  }
  h1 {
   text-align: center;
   color: white;
  }
  form {
   display: flex;
   justify-content: center;
   align-items: center;
   flex-direction: column;
   margin-top: 50px;
  }
  label {
   color: white;
   font-size: 24px;
   margin-bottom: 20px;
  }
  input {
   font-size: 20px;
   padding: 10px;
   width: 400px;
   border-radius: 5px;
   border: none;
   outline: none;
  }
  button {
   font-size: 20px;
   padding: 10px;
   background-color: #007bff;
   color: white;
   border: none;
   border-radius: 5px;
   cursor: pointer;
   transition: background-color 0.2s;
   margin-top: 20px;
  }
  button:hover {
   background-color: #0062cc;
  }
 </style>
</head>
<body>
 <h1>EV Charging Station Search Engine</h1>
 <form method="GET" action="/search">
  <label for="query">Enter search term:</label>
  <input type="text" name="query" id="query" placeholder="Enter search term...">
  <label for="search_type">Search by:</label>
  <select name="search_type" id="search_type">
    <option value="city">City</option>
    <option value="zip">Zip</option>
    <option value="ev_network">EV Network</option>
    <option value="ev_connector_types">Connector Types</option>
  </select>
  <button type="submit">Search</button>
 </form>
</body>
</html>
"""

with open('index.html','w') as f:
    f.write(html)

In [11]:
html_result = """
<!DOCTYPE html>
<html>
<head>
    <title>Search Result</title>
    <style>
        * {
            font-family: 'Montserrat', sans-serif;
            box-sizing: border-box;
        }

        body {
            background-color: #f5f5f5;
        }

        h1 {
            font-size: 3rem;
            margin: 2rem;
            text-align: center;
            color: #343a40;
        }

        .container {
            max-width: 1400px;
            margin: 0 auto;
            padding: 2rem;
            background-color: #fff;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.2);
        }

        table {
            width: 100%;
            border-collapse: collapse;
            margin-top: 2rem;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }

        th, td {
            border: 1px solid #ccc;
            padding: 1.2rem;
            text-align: center;
        }

        th {
            background-color: #f8f9fa;
            color: #6c757d;
            font-weight: 500;
            text-transform: uppercase;
        }

        tr:hover {
            background-color: #f2f2f2;
        }

        tr:nth-child(even) {
            background-color: #f8f9fa;
        }

        .highlight {
            color: #007bff;
            font-weight: 600;
        }

        @media (max-width: 767px) {
            h1 {
                font-size: 2.5rem;
            }

            table {
                font-size: 0.9rem;
            }
        }
    </style>
    <link href="https://fonts.googleapis.com/css?family=Montserrat:400,500,600&display=swap" rel="stylesheet">
</head>
<body>
    <div class="container">
        <h1>Charging Stations {% if search_type == "city" %}in city{% elif search_type == "zip" %}in zip{% elif search_type == "ev_network" %}in EV network{% elif search_type == "ev_connector_types" %}connector type{% else %}in{% endif %} 
        <span class="highlight">
        {% if search_type == "city" %}{{ result[0]['city'] }}{% endif %}
        {% if search_type == "zip" %}{{ result[0]['zip'] }}{% endif %}
        {% if search_type == "ev_network" %}{{ result[0]['ev_network'] }}{% endif %}
        {% if search_type == "ev_connector_types" %}{{ result[0]['ev_connector_types'] }}{% endif %}
        </span>
        </h1>
        <table>
            <thead>
                <tr>
                    <th>Station Name</th>
                    <th>Address</th>
                    <th>City</th>
                    <th>State</th>
                    <th>Zip</th>
                    <th>Station Phone</th>
                    <th>EV Network</th>
                    <th>Connector Types</th>
                </tr>
            </thead>
            <tbody>
                {% for document in result %}
                    <tr>
                        <td>{{ document['station_name'] }}</td>
                        <td>{{ document['street_address'] }}</td>
                        <td>{{ document['city'] }}</td>
                        <td>{{ document['state'] }}</td>
                        <td>{{ document['zip'] }}</td>
                        <td>{{ document['station_phone'] }}</td>
                        <td>{{ document['ev_network'] }}</td>
                        <td>{{ document['ev_connector_types'] }}</td>
                    </tr>
                {% endfor %}
            </tbody>
        </table>
    </div>
</body>
</html>
"""

with open('index_1.html','w') as f:
    f.write(html_result)

# Building Flask App

In [21]:
from flask import Flask, request, render_template, url_for
from pymongo import MongoClient

app = Flask(__name__)
client = MongoClient('localhost', 27017)
db = client['apan5400']
collection = db['APAN5400_Group_Demo']

@app.route('/')
def home():
    bg_image_url = url_for('static', filename='background.jpg')
    return render_template('index.html', bg_image_url=bg_image_url)

@app.route('/search', methods=['GET'])
def search():
    query = request.args.get('query')
    search_type = request.args.get('search_type')

    if search_type == 'zip':
        result = db.APAN5400_Group_Demo.find({"zip": {'$regex': query, '$options': 'i'}})
    elif search_type == 'city':
        result = db.APAN5400_Group_Demo.find({"city": {'$regex': query, '$options': 'i'}})
    elif search_type == 'ev_network':
        result = db.APAN5400_Group_Demo.find({"ev_network": {'$regex': query, '$options': 'i'}})
    elif search_type == 'ev_connector_types':
        result = db.APAN5400_Group_Demo.find({"ev_connector_types": {'$regex': query, '$options': 'i'}})
    else:
        result = db.APAN5400_Group_Demo.find()

    return render_template('index_1.html', result=result, search_type=search_type)

if __name__ == '__main__':
    app.run(port=5001)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Apr/2023 21:08:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2023 21:09:19] "GET /search?query=new+york&search_type=city HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2023 21:09:59] "GET /search?query=10023&search_type=zip HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2023 21:10:16] "GET /search?query=evgo&search_type=ev_network HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2023 21:10:38] "GET /search?query=Tesla&search_type=ev_connector_types HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2023 21:17:13] "GET /search?query=Tesla&search_type=ev_connector_types HTTP/1.1" 200 -


In [13]:
# Drop collection and Close connection
collection.drop()
client.close()